# Introduction

How much will a house in Kings County sell for? 

In this analysis, I will use data describing the physical properties and location of X homes in Kings County to show predict the price that similar houses will fetch in the real estate market.

## Methodology

### Exploring the data

To understand what we're looking at, I will first explore the data using visualizations to make explicit the relationships between the various characteristics of the homes.

### Describing the data

I will then make observations based on these visualizations and identify which factors are likely to be most predictive of house price.

### Defining the question

This analysis estimates both prediction and inference. What does that mean? I aim to _predict_ the selling price of comparable homes by creating a mathematical model informed by the data. I also aim to _infer_ the "why" behind that price. Thus, my model will answer questions like:

+ Which factors predict the outcome, and how? (e.g., are they positively or negatively correlated with price)
+ Which factors have the greatest impact on price? (e.g. size, age, waterfront view)
+ Is a particular house over- or under-valued relative to its peers?

### Preparing the data for analysis

I will clean and reformat the data to optimize it for creating a highly predictive and informative model.

### Fitting a model to the data

I use linear regression for this analysis for two reasons. First, it has high predictive value; it can be used to make assumptions about new homes, not just those in the dataset. Second, it is highly interpretable. It is not a "black box" algorithm; rather, a human being can inspect the reasons why I chose the factors I did to create the model. It will be easy to understand the intuitions that explain why any individual factor is predictive of the outcome.

In this section, I will use several different linear regression models to find the best fit. I will also verify their accuracy using statistical methods and visualizations.

### Visualizing the data

Finally, I will demonstrate the model's performance by providing visualizations of the predicted values and the actual observed values. 

# Exploring the Data

First, we'll need to import the data itself and some tools to help us analyze it. Below, I have imported several Python libraries for analyzing and visualizing data and loaded our data set. 

In [7]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as stats
import sklearn.linear_model as linear_model

import seaborn as sns

In [9]:
df = pd.read_csv('kc_house_data.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 76, saw 8
